In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

# 날짜 범위 설정
start_date = '2022-09-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# 월별 리밸런싱 수익률 계산 함수
def monthly_rebalance_return(df, weights):
    rebalance_returns = (df.pct_change().shift(-1) * weights).sum(axis=1)
    rebalance_cum_returns = (1 + rebalance_returns).cumprod()
    rebalance_cum_returns.iloc[0] = 1
    return rebalance_cum_returns

# MDD 계산 함수
def calculate_mdd(cum_returns):
    drawdown = cum_returns / cum_returns.cummax() - 1
    mdd = drawdown.cummin()
    return mdd

# 여러 종목 코드 그룹과 비중을 받아 개별 리밸런싱 결과를 그래프로 생성하는 함수
def plot_multiple_portfolios(single_stock_code, *multi_stock_codes_with_weights):
    # 한 종목의 데이터 가져오기
    single_stock_data = fdr.DataReader(single_stock_code, start_date, end_date).resample('ME').last()
    single_stock_cum_returns = (1 + single_stock_data['Close'].pct_change()).cumprod()
    single_stock_mdd = calculate_mdd(single_stock_cum_returns)

    # Plotly 그래프 생성 - 수익률 그래프
    fig_returns = go.Figure()
    fig_returns.add_trace(go.Scatter(
        x=single_stock_data.index,
        y=single_stock_cum_returns,
        name=f"{single_stock_code}",
        mode="lines+markers+text",
        text=single_stock_cum_returns.round(2),
        textposition="top center",
        texttemplate="%{text}"
    ))

    # Plotly 그래프 생성 - MDD 그래프
    fig_mdd = go.Figure()
    fig_mdd.add_trace(go.Scatter(
        x=single_stock_data.index,
        y=single_stock_mdd,
        name=f"{single_stock_code} MDD",
        mode="lines+markers+text",
        text=single_stock_mdd.round(2),
        textposition="bottom center",
        texttemplate="%{text}"
    ))

    # 각 그룹의 multi_stock_codes와 비중에 대해 개별적으로 수익률과 MDD 계산
    for i, (multi_stock_codes, weights) in enumerate(multi_stock_codes_with_weights, start=1):
        if len(multi_stock_codes) < 2:
            raise ValueError("각 multi_stock_codes 그룹에는 최소 2개 이상의 종목 코드가 필요합니다.")
        if len(weights) != len(multi_stock_codes):
            raise ValueError("비중 배열의 길이는 종목 코드의 개수와 같아야 합니다.")
        
        # 여러 종목의 데이터 가져오기
        multi_stock_data = {code: fdr.DataReader(code, start_date, end_date)['Close'] for code in multi_stock_codes}
        multi_stock_data = pd.DataFrame(multi_stock_data).resample('ME').last().dropna()
        
        # 비중을 np.array 형태로 변환 후 리밸런싱 포트폴리오 수익률 계산
        weights = np.array(weights)
        rebalance_cum_returns = monthly_rebalance_return(multi_stock_data, weights)
        rebalance_mdd = calculate_mdd(rebalance_cum_returns)
        
        # 리밸런싱 포트폴리오 수익률 추가
        fig_returns.add_trace(go.Scatter(
            x=rebalance_cum_returns.index,
            y=rebalance_cum_returns,
            name=f"Rebalanced Portfolio {i}",
            mode="lines+markers+text",
            text=rebalance_cum_returns.round(2),
            textposition="top center",
            texttemplate="%{text}"
        ))
        
        # 리밸런싱 포트폴리오 MDD 추가
        fig_mdd.add_trace(go.Scatter(
            x=rebalance_mdd.index,
            y=rebalance_mdd,
            name=f"Rebalanced Portfolio {i} MDD",
            mode="lines+markers+text",
            text=rebalance_mdd.round(2),
            textposition="bottom center",
            texttemplate="%{text}"
        ))

    # 그래프 레이아웃 설정
    fig_returns.update_layout(
        title="Monthly Cumulative Returns: Single Stock vs Multiple Rebalanced Portfolios",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Cumulative Returns")
    )
    
    fig_mdd.update_layout(
        title="Monthly MDD: Single Stock vs Multiple Rebalanced Portfolios",
        xaxis=dict(title="Date"),
        yaxis=dict(title="MDD")
    )
    
    fig_returns.show()
    fig_mdd.show()

# 예시 사용
single_stock_code = '379800'
multi_stock_codes_group1 = (['379800', '438080'], [0.7, 0.3])
multi_stock_codes_group2 = (['379800', '329750', '411060'], [0.5, 0.3, 0.2])
plot_multiple_portfolios(single_stock_code, multi_stock_codes_group1, multi_stock_codes_group2)
